## Zadanie domowe: morfologiczna gra w życie – John Conway

### Wykorzystanie operacji LUT w przekształceniu trafi, nie trafi
  - Szybszą metodą wykonania transformacji trafi, nie trafi może być operacja LUT.
  - Technika polega na zakodowaniu wyników wszystkich interesujących  konfiguracji, a następnie podczas przetwarzania wykorzystania operacji LUT.
  - Dla otoczenia 3x3 możliwe jest 512 różnych konfiguracji.
  - Aby praktycznie zrealizować operacje, każdej konfiguracji należy nadać unikalny indeks. Jedną z możliwości jest pomnożenie elementu strukturalnego przez macierz (mnożenie odpowiednich elementów):
  ```
  [[1, 8,  64],
   [ 2, 16, 128],
   [ 4, 32, 256]]
  ```
  Przykładowo elementowi:
  ```
  [[1, 1, 0],
   [ 1, 0, 1],
   [ 1, 0, 1]]
  ```
  odpowiada liczba: 1(1) + 2(1) + 4(1) + 8(1) + 128(1) + 256(1) = 399.
  
### Przykład działania metody – detekcja punktów końcowych na obrazie.
  - założenie: punkt końcowy to punkt, który ma dokładnie jednego sąsiada,
  - zdefiniuj funkcję, która jako argument pobiera otoczenie, a zwraca 0 lub 1 w zależności od tego, czy rozpatrywany punkt jest końcowy np. dla sąsiedztwa 3×3 punkt będzie końcowy, jeżeli jest zapalony i ma tylko jednego sąsiada (czyli suma pikseli jest równa 2).
  - wygeneruj przekodowanie LUT.
  - wczytaj obraz szkielet.bmp (należy go przekształcić, aby uzyskać dwuwymiarową tablicę o wartościach 0-1). Wykorzystując wygenerowane przekodowanie LUT wykonaj detekcję zakończeń. Wyświetl obraz oryginalny i po przekodowaniu LUT.

### Gra w życie

Reguły gry w życie:
  - każdy piksel biały, który ma dwóch lub trzech sąsiadów (białych) przeżywa,
  - każdy piksel biały, który ma 0,1 lub więcej niż trzech sąsiadów (białych) nie przeżywa (głód lub przeludnienie),
  - jeżeli jakieś pole (czarne) sąsiaduje dokładnie z trzema pikselami białymi, to na tym polu ,,rodzi'' się nowy piksel biały.

Zadanie:
  - za pomocą mechanizmu LUT (opisanego wcześniej) należy zaimplementować morfologiczną gre w życie,
  - najważniejszym elementem jest funkcja opisująca reguły gry,
  - symulacje należny przeprowadzić dla plansz dostarczonych w pliku gra.py,
  - dobrze jest wykonać kilka iteracji – zobaczyć jak zmienia się kształt,
  - inne ciekawe kształty do znalezienia w internecie.


In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

plansza1 = np.array([
    [0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0],
    [0,0,0,1,0,0,1,0,0,0],
    [0,0,0,1,1,1,1,0,0,0],
    [0,0,1,0,0,0,0,1,0,0],
    [0,0,1,0,1,1,0,1,0,0],
    [0,0,1,0,0,0,0,1,0,0],
    [0,0,0,1,1,1,1,0,0,0],
    [0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0]], bool)

plansza2 = np.array([
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0],
    [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0],
    [0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]], bool)

In [ ]:
LUT = np.array([[1<<0, 1<<3, 1<< 6], [1<<1, 1<<4, 1<<7], [1<<2, 1<<5, 1<<8]])
def iskonc(surr: np.ndarray) -> bool:
    return surr[surr.shape[0]//2, surr.shape[1]//2] and np.sum(surr) == 2

szkielet = cv2.imread("szkielet.bmp", cv2.IMREAD_GRAYSCALE)//255

plt.gray()
plt.figure()
plt.imshow(szkielet)

def generateiskonc():
    out = []
    for i in range(512):
        if iskonc(np.array([[int(i & 1), int(i & 2) // 2, int(i & 4) // 4],
                           [int(i & 8) //8, int(i & 16) // 16, int(i & 32) // 32],
                           [int(i & 64) // 64, int(i & 128) // 128, int(i & 256) // 256]])):
            out.append(1)
        else:
            out.append(0)
    return out

def detect_iskonc(img: np.ndarray):
    out = np.zeros(img.shape)
    stt = generateiskonc()
    for row in range(1, img.shape[0]-1):
        for col in range(1, img.shape[1]-1):
            surr = img[row-1:row+2, col-1:col+2]
            if surr.shape == (3, 3):
                out[row, col] = stt[np.sum(np.multiply(LUT, surr))]
    return out

plt.figure()
plt.imshow(detect_iskonc(szkielet))
#No takie średnie to wykrywanie końcówek

In [ ]:
def white_LUT():
    lut = np.zeros(512, dtype=bool)
    for i in range(512):
        sss = np.sum([[int(i & 1), int(i & 2) // 2, int(i & 4) // 4],
                           [int(i & 8) //8, int(i & 16) // 16, int(i & 32) // 32],
                           [int(i & 64) // 64, int(i & 128) // 128, int(i & 256) // 256]])
        if 3 >= sss - 1 >= 2:
            lut[i] = True
    return lut

def black_LUT():
    lut = np.zeros(512, dtype=bool)
    for i in range(512):
        sss = np.sum([[int(i & 1), int(i & 2) // 2, int(i & 4) // 4],
                           [int(i & 8) //8, int(i & 16) // 16, int(i & 32) // 32],
                           [int(i & 64) // 64, int(i & 128) // 128, int(i & 256) // 256]])
        if sss == 3:
            lut[i] = True
    return lut

def game(plansza: np.ndarray, cycles: int):
    whiteLUT = white_LUT()
    blackLUT = black_LUT()
    while cycles > 0:
        new_plansza = np.zeros(plansza.shape)
        cycles -= 1
        for row in range(1, plansza.shape[0]-1):
            for col in range(1, plansza.shape[1]-1):
                surr = plansza[row-1:row+2, col-1:col+2]
                if plansza[row, col]:
                    new_plansza[row, col] = whiteLUT[np.sum(np.multiply(LUT, surr), dtype=np.uint16)]
                else:
                    new_plansza[row, col] = blackLUT[np.sum(np.multiply(LUT, surr), dtype=np.uint16)]
        plansza = new_plansza
    return plansza[1:plansza.shape[0]-1, 1:plansza.shape[1]-1]


for i in range(10):
    plt.figure()
    plt.imshow(game(plansza2, i))